# Exploring the Capabilities of LLM Models

In this notebook, I aim to evaluate and compare the capabilities of two large language models (LLMs):

1. **Codestral22B**
   A state-of-the-art model designed for advanced code generation and natural language understanding tasks.

2. **Llama 3.1-8B**
   A highly efficient and compact model optimized for general-purpose language tasks with an 8-billion parameter architecture.

The goal is to analyze their performance across various tasks, including but not limited to:

- Code generation and completion
- Natural language understanding
- Contextual reasoning
- Problem-solving capabilities

This comparison will help identify the strengths and weaknesses of each model and provide insights into their practical applications.

# AI-Powered Programming Tutor with RAG

This project focuses on building an AI-powered programming tutor designed to assist students in understanding code and solving problems. The tutor leverages **Retrieval-Augmented Generation (RAG)** to provide accurate and personalized explanations grounded in real university materials, such as:

- Past assignments
- Lecture notes
- Tutorials`

The system integrates two large language models (LLMs), **Codestral22B** and **Llama 3.1-8B**, to evaluate their performance in generating solutions and explanations for programming-related queries. The goal is to determine which model provides better support for students in a university setting.

---

## Key Features

- **Personalized Explanations**: Tailored responses based on retrieved university materials.
- **Code Understanding**: Helps students debug and understand code snippets.
- **Problem Solving**: Provides step-by-step solutions to programming problems.
- **Model Comparison**: Evaluates the performance of Codestral22B and Llama 3.1-8B.

---



### Basically, I will try to implement and test Codestral22B and Llama 3.1-8B